
# Fibonacci‑Thread Mega‑Suite (v2)

*Autogenerated 2025-07-20 10:48 UTC. Self‑contained, Colab‑ready, no uploads required.*

**Purpose** — stress‑test the hypothesis that the Fibonacci binary mask is the “perfectly‑taught thread” by running a **battery of symmetry and coupling diagnostics** across multiple prefix sizes, prime ranges, and control sequences.  

Each analytic lens runs **three passes** for deterministic robustness; toggle `RUN_CONTROLS` / `DEEP_MODE` in the config to widen or shrink workload.


In [1]:

import numpy as np, mpmath as mp, matplotlib.pyplot as plt, time, math, hashlib, itertools, random
mp.dps = 50


In [2]:

# ================= CONFIG SECTION =================
FIB_LEN_LIST       = [300_000, 1_000_000]      # prefix lengths to scan
PRIME_CUTOFF_LIST  = [5_000_000, 25_000_000]
SHIFT_MAX          = 12_000        # for autocorr heat‑map
MOD_LIST           = [3,5,7,9,11,13,17,19,23,29]
HAAR_LEVELS        = 6
THETA_TEST_LIST    = [0.10,0.11,0.12,0.13,0.14]
RUN_CONTROLS       = True         # random & periodic sequences; flip True for full run
DEEP_MODE          = False         # True → run every prefix × prime‑cutoff combo
np.random.seed(42)


In [3]:

################# Utility functions #################

def fibonacci_word(n:int):
    a,b="0","01"
    while len(b)<n:
        a,b=b,a+b
    return b[:n]

_sieve_cache={}
def primes_upto(N:int):
    if N in _sieve_cache: return _sieve_cache[N]
    sieve = bytearray(b"\x01")*(N+1)
    sieve[0:2]=b"\x00\x00"
    for p in range(2,int(N**0.5)+1):
        if sieve[p]:
            sieve[p*p:N+1:p]=b"\x00"*(((N-p*p)//p)+1)
    _sieve_cache[N]=[p for p in range(2,N+1) if sieve[p]]
    return _sieve_cache[N]

def autocorr_heat(bits, shift_max):
    n=len(bits)-shift_max
    xor=np.empty(shift_max)
    for k in range(1,shift_max+1):
        xor[k-1]=(bits[:n] ^ bits[k:k+n]).mean()
    return xor

def prime_overlap_discrepancy(bits, primes, mod):
    mask_pr=np.zeros_like(bits)
    for p in primes:
        if p>len(bits): break
        mask_pr[p-1]=1
    overlap=(bits & mask_pr)
    idx=np.arange(len(bits))
    counts=np.bincount(idx%mod, weights=overlap, minlength=mod)
    expected=overlap.sum()/mod
    return np.abs(counts-expected)

def haar_energy(bits, level):
    w=3**level
    blocks=len(bits)//w
    if blocks==0: return 0.0
    e=0.0
    for b in range(blocks):
        seg=bits[b*w:(b+1)*w]
        thirds=np.split(seg,3)
        avg=[s.mean() for s in thirds]
        e+=(avg[0]-avg[1])**2 + (avg[1]-avg[2])**2 + (avg[0]-avg[2])**2
    return e/blocks

def psi_weighted(theta,x,primes):
    tot=0
    for p in primes:
        if p>x: break
        k=1
        while p**k<=x and k<=3:
            tot+=(x//p**k)*p**k*(1-p**(-k*theta))
            k+=1
    return tot

def generate_control(kind,n):
    if kind=='random':
        return np.random.randint(0,2,size=n,dtype=np.uint8)
    if kind=='period34':
        base=np.array(([1]+[0]*2)*11+[1]+[0],dtype=np.uint8)  # 34‑period pattern 100... sequence
        reps=(n//34)+1
        return np.tile(base,reps)[:n]
    if kind=='sturm_sqrt2':
        alpha=mp.sqrt(2)-1
        return np.fromiter(((int((k+1)*alpha)%2) for k in range(n)),dtype=np.uint8,count=n)
    raise ValueError("unknown control")


In [4]:

################# Master experiment suite #################

def run_suite(bits,label,primes):
    if isinstance(bits,str):
        arr=np.frombuffer(bits.encode(),dtype=np.uint8)-48
    else:
        arr=bits
    print(f"\n=== SUITE {label} ===")
    # 1. Autocorr (3 passes)
    for p in range(3):
        ac=autocorr_heat(arr, SHIFT_MAX)
        print(f"Autocorr pass {p+1}: min={ac.min():.6f} at shift {ac.argmin()+1}")
    # 2. Prime discrepancy
    for m in MOD_LIST:
        disp=prime_overlap_discrepancy(arr,primes,m)
        print(f"mod{m}: max{disp.max():.0f}", end=' | ')
    print()
    # 3. Wavelet
    energies=[haar_energy(arr,l) for l in range(1,HAAR_LEVELS+1)]
    print("Wavelet energies:",["{:.3e}".format(e) for e in energies])
    # 4. Theta coupling quick scan
    for theta in THETA_TEST_LIST:
        raw=[psi_weighted(theta,x,primes)-x for x in (1_000_000,2_000_000,5_000_000)]
        bias=np.mean(raw)
        resid=np.mean([abs(r-bias) for r in raw])
        print(f"θ={theta:.2f} resid {resid:.3e}")


In [5]:

def master():
    start=time.time()
    for fib_len in FIB_LEN_LIST:
        fib=fibonacci_word(fib_len)
        for pcut in PRIME_CUTOFF_LIST:
            primes=primes_upto(pcut)
            run_suite(fib,f'Fib L={fib_len} Pcut={pcut}',primes)
            if not DEEP_MODE: break
        if not DEEP_MODE: break

    if RUN_CONTROLS:
        n=FIB_LEN_LIST[0]
        primes=primes_upto(PRIME_CUTOFF_LIST[0])
        for kind in ['random','period34','sturm_sqrt2']:
            seq=generate_control(kind,n)
            run_suite(seq,kind,primes)
    print("Total elapsed:",time.time()-start,"s")

# --- Execute ---
master()



=== SUITE Fib L=300000 Pcut=5000000 ===
Autocorr pass 1: min=0.000076 at shift 10946
Autocorr pass 2: min=0.000076 at shift 10946
Autocorr pass 3: min=0.000076 at shift 10946
mod3: max3281 | mod5: max1968 | mod7: max1407 | mod9: max1094 | mod11: max895 | mod13: max756 | mod17: max579 | mod19: max518 | mod23: max427 | mod29: max339 | 
Wavelet energies: ['2.000e+00', '9.726e-02', '2.469e-02', '2.577e-03', '5.558e-05', '1.854e-05']
θ=0.10 resid 4.354e+11
θ=0.11 resid 4.535e+11
θ=0.12 resid 4.693e+11
θ=0.13 resid 4.829e+11
θ=0.14 resid 4.947e+11

=== SUITE random ===
Autocorr pass 1: min=0.496333 at shift 3528
Autocorr pass 2: min=0.496333 at shift 3528
Autocorr pass 3: min=0.496333 at shift 3528
mod3: max4313 | mod5: max2588 | mod7: max1849 | mod9: max1438 | mod11: max1177 | mod13: max996 | mod17: max761 | mod19: max680 | mod23: max562 | mod29: max446 | 
Wavelet energies: ['1.500e+00', '4.987e-01', '1.682e-01', '5.627e-02', '1.779e-02', '6.198e-03']
θ=0.10 resid 4.354e+11
θ=0.11 resid 4.

In [6]:
print('Notebook fingerprint:', hashlib.sha256(b'FibMegaSuiteV2').hexdigest()[:16])

Notebook fingerprint: 4ae016243e02b6eb
